<a href="https://colab.research.google.com/github/bsivavenu/ML-A-Z/blob/master/multi_lable_movie_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz 
# from this link you can download the dataset and play

In [126]:
ls

character.metadata.tsv  name.clusters.txt   README.txt
movie.metadata.tsv      plot_summaries.txt  tvtropes.clusters.txt


In [0]:
!tar -xvf MovieSummaries.tar.gz

MovieSummaries/
MovieSummaries/tvtropes.clusters.txt
MovieSummaries/name.clusters.txt
MovieSummaries/plot_summaries.txt
MovieSummaries/README.txt
MovieSummaries/movie.metadata.tsv
MovieSummaries/character.metadata.tsv


In [0]:
pwd()

'/content'

In [0]:
cd MovieSummaries/

/content/MovieSummaries


In [0]:
ls

character.metadata.tsv  name.clusters.txt   README.txt
movie.metadata.tsv      plot_summaries.txt  tvtropes.clusters.txt


In [0]:
import pandas as pd
import json
import numpy as np
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [0]:
meta = pd.read_csv("movie.metadata.tsv", sep = '\t', header = None)
meta.head()

,0,1,2,3,4,5,6,7,8
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [0]:
meta.columns = ["movie_id",1,"movie_name",3,4,5,6,7,"genre"]


In [0]:
meta.head()

,movie_id,1,movie_name,3,4,5,6,7,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [0]:
plots = []

with open("plot_summaries.txt", 'r') as f:
       reader = csv.reader(f, dialect='excel-tab') 
       for row in tqdm(reader):
            plots.append(row)

42303it [00:01, 37047.29it/s]


In [127]:
plots[:2]

[['23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."],
 ['31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at speci

In [0]:
df = pd.read_csv('plot_summaries.txt',delimiter='\t',header=None)
df.head()

,0,1
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


In [0]:
df.columns = ['movie_id','plot']
df.head(2)

,movie_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...


In [0]:
meta.head(2)

,movie_id,1,movie_name,3,4,5,6,7,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


In [0]:
movies = pd.merge(df,meta[['movie_id','movie_name','genre']],on='movie_id')
movies.head(2)

,movie_id,plot,movie_name,genre
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci..."
1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":..."


In [0]:
movies['genre'][0]

'{"/m/07s9rl0": "Drama", "/m/03q4nz": "World cinema"}'

In [0]:
geners = []

for i in movies['genre']:
  geners.append(list(json.loads(i).values()))


geners[:5]

[['Drama', 'World cinema'],
 ['Action/Adventure', 'Science Fiction', 'Action', 'Drama'],
 ['Musical', 'Action', 'Drama', 'Bollywood'],
 ['Screwball comedy', 'Comedy'],
 ['Crime Fiction', 'Drama', 'Docudrama', 'World cinema', 'Courtroom Drama']]

In [0]:
movies['new_geners'] = geners


In [0]:
movies.head(2)

,movie_id,plot,movie_name,genre,gener_new,new_geners
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...",None,"[Drama, World cinema]"
1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...",None,"[Action/Adventure, Science Fiction, Action, Dr..."


In [0]:
len(movies[movies['new_geners'].str.len()==0])

411

In [0]:
movies_new = movies[~(movies['new_geners'].str.len()==0)]


In [0]:
movies.shape,movies_new.shape

((42204, 6), (41793, 6))

In [0]:
movies_new.head(2)

,movie_id,plot,movie_name,genre,gener_new,new_geners
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...",None,"[Drama, World cinema]"
1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...",None,"[Action/Adventure, Science Fiction, Action, Dr..."


In [0]:
# del movies_new['genre']
del movies_new['gener_new']


In [0]:
movies_new.head(2)

,movie_id,plot,movie_name,new_geners
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"[Drama, World cinema]"
1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"[Action/Adventure, Science Fiction, Action, Dr..."


In [0]:
all_genres = sum(geners,[])


150775

In [0]:
len(set((all_genres)))

363

In [0]:
all_genres = nltk.FreqDist(all_genres) 

# create dataframe
all_genres_df = pd.DataFrame({'Genre': list(all_genres.keys()), 
                              'Count': list(all_genres.values())})

In [0]:
all_genres_df.head()

,Genre,Count
0,Drama,19134
1,World cinema,5153
2,Action/Adventure,3553
3,Science Fiction,2339
4,Action,5868


In [0]:
def clean_text(text):
  text = re.sub("\'",'',text)
  text = re.sub("[^a-zA-Z]"," ",text) 
  text = ' '.join(text.split())
  text = text.lower()
  return text

In [0]:
movies_new['clean_plot'] = movies_new['plot'].apply(lambda x: clean_text(x) )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
movies_new['clean_plot'][:5]

0    shlykov a hard working taxi driver and lyosha ...
1    the nation of panem consists of a wealthy capi...
2    poovalli induchoodan is sentenced for six year...
3    the lemon drop kid a new york city swindler is...
4    seventh day adventist church pastor michael ch...
Name: clean_plot, dtype: object

In [0]:
import nltk
nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


In [0]:

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

movies_new['clean_plot'] = movies_new['clean_plot'].apply(lambda x: remove_stopwords(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [0]:
movies_new['clean_plot'][:5]

0    shlykov hard working taxi driver lyosha saxoph...
1    nation panem consists wealthy capitol twelve p...
2    poovalli induchoodan sentenced six years priso...
3    lemon drop kid new york city swindler illegall...
4    seventh day adventist church pastor michael ch...
Name: clean_plot, dtype: object

In [0]:
from sklearn.preprocessing import  MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()

In [0]:
y = multilabel_binarizer.fit_transform(movies_new['new_geners'])


In [0]:
len(y[0])

363

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [0]:
xtrain, xval, ytrain, yval = train_test_split(movies_new['clean_plot'], y, test_size=0.2, random_state=9)

In [0]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [0]:
type(xtrain_tfidf)

scipy.sparse.csr.csr_matrix

In [0]:
from sklearn.linear_model import LogisticRegression
# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [0]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [0]:
clf.fit(xtrain_tfidf, ytrain)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 48 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 182 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [0]:
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)

In [0]:
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
multilabel_binarizer.inverse_transform(y_pred)[1]

('Comedy', 'Family Film')

In [0]:
f1_score(yval, y_pred, average="micro")

0.31539641943734015

In [0]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [0]:
for i in range(5): 
  k = xval.sample(1).index[0] 
  print("Movie: ", movies_new['movie_name'][k], "\nPredicted genre: ", infer_tags(xval[k])), print("Actual genre: ",movies_new['new_geners'][k], "\n")


Movie:  Godzilla vs. Gigan 
Predicted genre:  [('Fantasy', 'Japanese Movies', 'Monster', 'Science Fiction', 'World cinema')]
Actual genre:  ['Science Fiction', 'Japanese Movies', 'World cinema', 'Creature Film', 'Cult', 'Fantasy', 'Monster'] 

Movie:  Trash 
Predicted genre:  [('Comedy', 'Drama')]
Actual genre:  ['LGBT', 'Crime Fiction', 'Drama', 'Indie', 'Gay Themed'] 

Movie:  Hapkido 
Predicted genre:  [()]
Actual genre:  ['Action/Adventure', 'Action', 'Martial Arts Film', 'Chinese Movies', 'World cinema'] 

Movie:  Shottas 
Predicted genre:  [('Action', 'Crime Fiction', 'Drama', 'Thriller')]
Actual genre:  ['Crime Fiction', 'Gangster Film', 'Action', 'Drama', 'Crime Drama'] 

Movie:  Slayer 
Predicted genre:  [('Action', 'Action/Adventure')]
Actual genre:  ['Action', 'Horror', 'Indie', 'Television movie'] 

